In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from tqdm import tqdm

from scipy import sparse

In [76]:
import json

In [111]:
X_train = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_train.npz')

In [23]:
cc= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_train.npy")

In [7]:
model = LatentDirichletAllocation(n_components = 25, verbose=1)

In [8]:
model.fit(X_train)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=25, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [9]:
from joblib import dump, load
dump(model, 'lda.joblib') 

['lda.joblib']

In [96]:
X_val = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_val.npz')
X_test = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_test.npz')

total_X = sparse.vstack([X_val, X_test])

ix = total_X.getnnz(1)>0
total_X = total_X[ix]

In [14]:
transformed = model.transform(total_X)

In [19]:
train_transform = model.transform(X_train)
np.save("lda_transformed_train", train_transform)

In [15]:
import numpy as np

np.save("lda_transformed_test", transformed)

In [18]:
np.sum(transformed, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [20]:
from sklearn.linear_model import LogisticRegression 

In [29]:
logit = LogisticRegression(solver = 'liblinear', verbose= 2)

In [30]:
logit.fit(train_transform, cc)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=2,
                   warm_start=False)

In [31]:
dump(logit, "lda_reduced_trained_logit")

['lda_reduced_trained_logit']

In [32]:
predictions = logit.predict(transformed)

In [41]:
np.save("logit_predictions_lda_reduced", predictions)

In [33]:
from sklearn.metrics import classification_report

In [34]:
cc_val= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_val.npy")

In [35]:
cc_test= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_test.npy")

In [36]:
total_c = np.concatenate((cc_val, cc_test))
total_c= total_c[ix]

In [64]:
print(classification_report(total_c, predictions))

              precision    recall  f1-score   support

           0       0.11      0.00      0.00     31405
           1       0.60      0.61      0.60    113782
           2       0.21      0.05      0.08     25626
           3       0.00      0.00      0.00      6474
           4       0.49      0.54      0.52    141680
           5       0.00      0.00      0.00     17033
           6       0.00      0.00      0.00      8933
           7       0.00      0.00      0.00      6293
           8       0.00      0.00      0.00      9964
           9       0.25      0.08      0.12     26425
          10       0.32      0.60      0.42    271113
          11       0.33      0.36      0.34     94169
          12       0.31      0.00      0.00     11026
          13       0.00      0.00      0.00      7135
          14       0.46      0.48      0.47     27818
          15       0.31      0.34      0.32    263881
          16       0.20      0.02      0.04     35477
          17       0.00    

### c3rLDA 

In [42]:
logitc3 = LogisticRegression(solver = 'liblinear', verbose= 1)

In [51]:
c3_ndz = np.load("c3rLDA_eval_ndz.npy")

In [52]:
c3_ndz= (c3_ndz + .1).astype(float)
c3_ndz /= np.sum(c3_ndz, axis=1)[:, np.newaxis]

In [55]:
c3_train_ndz = np.load("./model_persistence/all_eths_less_than_15000_1000_its_25_topics/ndz.npy")

In [56]:
c3_train_ndz= (c3_train_ndz + .1).astype(float)
c3_train_ndz /= np.sum(c3_train_ndz, axis=1)[:, np.newaxis]

In [59]:
ix = X_train.getnnz(1)>0
cc= cc[ix]

In [70]:
X_train = X_train[ix]

In [60]:
logitc3.fit(c3_train_ndz, cc) 

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [61]:
predictions_c3 = logitc3.predict(c3_ndz)

In [63]:
print(classification_report(total_c, predictions_c3))

              precision    recall  f1-score   support

           0       0.13      0.00      0.00     31405
           1       0.20      0.13      0.16    113782
           2       0.00      0.00      0.00     25626
           3       0.00      0.00      0.00      6474
           4       0.20      0.02      0.04    141680
           5       0.00      0.00      0.00     17033
           6       0.00      0.00      0.00      8933
           7       0.00      0.00      0.00      6293
           8       0.00      0.00      0.00      9964
           9       0.00      0.00      0.00     26425
          10       0.26      0.63      0.37    271113
          11       0.25      0.43      0.31     94169
          12       0.00      0.00      0.00     11026
          13       0.00      0.00      0.00      7135
          14       0.00      0.00      0.00     27818
          15       0.25      0.37      0.30    263881
          16       0.15      0.00      0.01     35477
          17       0.00    

## ccLDA

In [65]:
logitcc = LogisticRegression(solver = 'liblinear', verbose= 1)
cc_train_ndz = np.load("./cc_ndz.npy")

In [66]:
cc_train_ndz = (cc_train_ndz + .1).astype(float)
cc_train_ndz /= np.sum(cc_train_ndz, axis=1)[:, np.newaxis]

In [67]:
logitcc.fit(cc_train_ndz, cc) 

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [68]:
cc_ndz = np.load("c3rLDA_eval_ndz.npy")
cc_ndz= (cc_ndz + .1).astype(float)
cc_ndz /= np.sum(cc_ndz, axis=1)[:, np.newaxis]


predictions_cc = logitcc.predict(cc_ndz)

In [69]:
print(classification_report(total_c, predictions_cc))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     31405
           1       0.05      0.08      0.06    113782
           2       0.00      0.00      0.00     25626
           3       0.00      0.00      0.00      6474
           4       0.06      0.00      0.00    141680
           5       0.00      0.00      0.00     17033
           6       0.00      0.00      0.00      8933
           7       0.00      0.00      0.00      6293
           8       0.00      0.00      0.00      9964
           9       0.00      0.00      0.00     26425
          10       0.16      0.33      0.22    271113
          11       0.03      0.00      0.00     94169
          12       0.00      0.00      0.00     11026
          13       0.00      0.00      0.00      7135
          14       0.00      0.00      0.00     27818
          15       0.17      0.22      0.19    263881
          16       0.00      0.00      0.00     35477
          17       0.00    

## No dimensionality reduction

In [71]:
logitno = LogisticRegression(solver = 'liblinear', verbose= 1)
logitno.fit(X_train, cc) 

[LibLinear]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [72]:
predictions_no = logitno.predict(total_X)

In [73]:
print(classification_report(total_c, predictions_no))

              precision    recall  f1-score   support

           0       0.37      0.16      0.23     31405
           1       0.71      0.70      0.71    113782
           2       0.55      0.41      0.47     25626
           3       0.82      0.51      0.63      6474
           4       0.73      0.65      0.69    141680
           5       0.59      0.35      0.44     17033
           6       0.58      0.31      0.40      8933
           7       0.22      0.03      0.06      6293
           8       0.17      0.05      0.08      9964
           9       0.60      0.38      0.46     26425
          10       0.42      0.63      0.50    271113
          11       0.63      0.58      0.60     94169
          12       0.65      0.40      0.50     11026
          13       0.66      0.29      0.41      7135
          14       0.61      0.61      0.61     27818
          15       0.41      0.47      0.44    263881
          16       0.68      0.42      0.52     35477
          17       0.12    

## Linear regression on Locations

In [85]:
locs = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_train.json", 'r') as file:
    locs = json.load(file)

In [86]:
locs = np.array(locs)

In [80]:
ix = X_train.getnnz(1)>0
locs= np.array(locs)[ix]

In [84]:
X_train = X_train[ix]

In [81]:
from sklearn.linear_model import LinearRegression

In [83]:
lda_reg = LinearRegression()

In [89]:
lda_reg.fit(train_transform, locs)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [90]:
lda_reg_pred = lda_reg.predict(transformed)

In [99]:
locs_val = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_val.json", 'r') as file:
    locs_val = json.load(file)
locs_test = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_test.json", 'r') as file:
    locs_test = json.load(file)
    
total_locs = locs_val + locs_test
total_locs= np.array(total_locs)[ix]

In [127]:
import geopy.distance

distance_errors = []

for i in range(total_locs.shape[0]):
    distance_errors.append(geopy.distance.distance(total_locs[i], lda_reg_pred[i]).km)

In [102]:
sum(distance_errors)

TypeError: unsupported operand type(s) for +: 'int' and 'geodesic'

In [108]:
sum([x.km for x in distance_errors])/len(distance_errors)

1335.734786532042

## Linear regression for c3rlda

In [109]:
c3rlda_reg = LinearRegression()

In [112]:
ix = X_train.getnnz(1)>0
locs = locs[ix]

In [113]:
c3rlda_reg.fit(c3_train_ndz, locs)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [116]:
total_locs.shape

(1430699, 2)

In [114]:
c3rlda_reg_pred = c3rlda_reg.predict(c3_ndz)

c3_distance_errors = []



In [117]:
for i in range(total_locs.shape[0]):
    c3_distance_errors.append(geopy.distance.distance(total_locs[i], c3rlda_reg_pred[i]).km)


TypeError: unsupported operand type(s) for +: 'int' and 'geodesic'

In [120]:
sum(c3_distance_errors)/len(c3_distance_errors)

1341.0551765832804

## Linear regression for ccLDA

In [121]:
cclda_reg = LinearRegression()
cclda_reg.fit(cc_train_ndz, locs)
cclda_reg_pred = cclda_reg.predict(cc_ndz)

In [122]:
cc_distance_errors = []

for i in range(total_locs.shape[0]):
    cc_distance_errors.append(geopy.distance.distance(total_locs[i], cclda_reg_pred[i]).km)

print(sum(cc_distance_errors)/len(cc_distance_errors))

1392.66273026113


In [123]:
import statistics

In [124]:
statistics.median(cc_distance_errors)

1266.9376946555874

In [125]:
statistics.median(c3_distance_errors)

1227.7392804898634

In [128]:
statistics.median(distance_errors)

1234.425078109626